In [2]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import requests
import pprint
from bs4 import BeautifulSoup
import datetime
import matplotlib.pyplot as plt

In [3]:
client = MongoClient('10.52.51.28', 27017).ourinhos.bitcoin
url = 'https://www.infomoney.com.br/cryptos/cotacoes'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
crypt = soup.find_all(class_ = "ticker-internal")

In [4]:
client.delete_many({})

In [5]:
for coin in crypt:
    titulo = str(coin.find(class_ = "crypto__ticker-item-logo")).split('title')[1].replace('="', '').replace('"/>', '')
    
    if (titulo != 'Conheça mais sobre a Xdex'):
        dh_pesquisa = datetime.datetime.now()
        nm_moeda = coin.find('a').text
        pc_percentual = coin.find(class_ = 'ticker-box-positive').text
        vr_valor = coin.find(class_ = "value").text
 
        insert = {
            'dh_pesquisa': dh_pesquisa.strftime("%Y-%m-%d %H:%M"),
            'nm_moeda': nm_moeda,
            'pc_percentual': float(pc_percentual.replace('%', '').replace('+', '').replace(',', '.')),
            'vr_valor': float(vr_valor.split()[1].replace('%', '').replace('+', '').replace('.', '').replace(',', '.'))}
    
        client.insert_one(insert).inserted_id

In [7]:
moeda = []
percentual = []
valor = []

client = MongoClient('10.52.51.28', 27017).ourinhos.bitcoin
for document in client.find():
    moeda.append(document['nm_moeda'])
    percentual.append(document['pc_percentual'])
    valor.append(document['vr_valor'])
client.close

Collection(Database(MongoClient(host=['10.52.51.28:27017'], document_class=dict, tz_aware=False, connect=True), 'ourinhos'), 'bitcoin.close')

In [8]:
moeda

['BITCOIN',
 'ETHEREUM',
 'RIPPLE',
 'EOS',
 'LITECOIN',
 'BITCOIN CASH',
 'TETHER',
 'STELLAR']